# ROPE Full Evaluation on Google Colab

Run a full or custom ROPE evaluation on Colab GPU and download all outputs.

**Runtime:** Set to **GPU** (A100 preferred for full run; T4 works for smaller runs).

**Rough times (T4):**
- Quick: 1 model, 2 defenses, 20 attacks -> ~15 min
- Medium: 2 models, 2 defenses, 60 attacks -> ~2-3 h
- Full: 3 models, 4 defenses, 120 attacks -> ~12-24 h (use A100 or run overnight)

## 1. Setup (clone or use uploaded ROPE)

In [ ]:
import os
import torch

print(f"GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'None'}")

REPO_URL = "https://github.com/yourusername/ROPE.git"
REPO_DIR = "ROPE"

if not os.path.isdir(REPO_DIR) and "yourusername" not in REPO_URL:
    !git clone --depth 1 "{REPO_URL}" "{REPO_DIR}"

if os.path.isdir(REPO_DIR):
    %cd {REPO_DIR}
    !pip install -e . -q
    print("ROPE ready.")
else:
    print("Upload ROPE zip in colab_rope_demo.ipynb first, then run from that folder.")

## 2. HuggingFace login

In [ ]:
from getpass import getpass
from huggingface_hub import login

token = getpass("HuggingFace token (Enter to skip): ")
if token.strip():
    login(token=token.strip())
    print("Logged in.")

## 3. Choose and run evaluation

Uncomment ONE block below. Key flags:
- `--verbose`: save full debug info (judge outputs, defended prompts)
- `--judge phi2`: use phi2 as judge (faster, less accurate)
- `--judge llama3-8b`: use llama3-8b as judge (default, more accurate)
- `--resume`: continue from where you left off if the run crashed

In [ ]:
# --- Quick (~15 min T4): 1 model, 2 defenses, 20 attacks ---
# import json
# with open('data/attacks.json') as f: a = json.load(f)
# with open('data/attacks_quick.json','w') as f: json.dump(a[:20], f)
# !rope run --models phi2 --defenses none,delimiter --attacks data/attacks_quick.json --judge phi2 --verbose -o quick_results.json

# --- Medium (~2h T4): 2 models, 2 defenses, 60 attacks ---
# import json
# with open('data/attacks.json') as f: a = json.load(f)
# with open('data/attacks_60.json','w') as f: json.dump(a[:60], f)
# !rope run --models phi2,llama2-7b --defenses none,delimiter --attacks data/attacks_60.json --verbose -o medium_results.json

# --- Full (12-24h A100): 3 models, 4 defenses, 120 attacks ---
!rope run --models llama2-7b,llama3-8b,phi2 --defenses none,delimiter,icl,paraphrase --verbose -o full_results.json

## 4. View summary and download outputs

In [ ]:
import os
import json
import pandas as pd

# Use the output name you chose above
OUT = "full_results"  # or quick_results / medium_results
json_path = OUT + ".json"
csv_path = OUT + "_metrics.csv"
report_path = OUT + "_report.txt"

if os.path.exists(json_path):
    results = json.load(open(json_path))

    # Overall metrics
    if os.path.exists(csv_path):
        metrics = pd.read_csv(csv_path)
    else:
        from rope.metrics import compute_metrics
        metrics = compute_metrics(results)

    print("OVERALL METRICS")
    display(metrics)

    # Per-attack-type breakdown
    from rope.metrics import compute_by_attack_type
    by_type = compute_by_attack_type(results)
    print("\nBY ATTACK TYPE")
    display(by_type)

    # Report
    if os.path.exists(report_path):
        print("\n" + open(report_path).read())

    # Severity distribution
    from collections import Counter
    sev_dist = Counter(r['severity'] for r in results)
    print(f"\nSeverity distribution: {dict(sorted(sev_dist.items()))}")
    print(f"Total evaluations: {len(results)}")
else:
    print(f"No {json_path} yet. Run the evaluation cell above.")

In [ ]:
from google.colab import files

for name in [OUT + ".json", OUT + "_metrics.csv", OUT + "_report.txt"]:
    if os.path.exists(name):
        files.download(name)
        print(f"Downloaded {name}")